In [4]:
import requests
import pandas as pd

import sys

sys.path.append("S://Docs//Personal//MAEVE//Data//Config//")

from config import *

## Get Historical data

In [3]:
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=60')

df = pd.DataFrame(resp.json()['result']['XXBTZUSD'], columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'])
df['time'] = pd.to_datetime(df['time'], unit='s')

print(df.shape)

(720, 8)


## Get price

In [4]:
import requests

url = "https://api.kraken.com/0/public/Ticker?pair=XBTUSD"

response = requests.get(url)
data = response.json()

price = data['result']['XXBTZUSD']['c'][0]
print("Current price of BTC:", price)

Current price of BTC: 23387.00000


## Get USD balance

In [3]:
import time
import os
import requests

import urllib.parse
import hashlib
import hmac
import base64


def get_kraken_signature(urlpath, data, secret):

    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()



# Read Kraken API key and secret stored in environment variables
api_url = "https://api.kraken.com"

# Attaches auth headers and returns results of a POST request


def kraken_request(uri_path, data):
    headers = {}
    headers['API-Key'] = API_KEY
    # get_kraken_signature() as defined in the 'Authentication' section
    headers['API-Sign'] = get_kraken_signature(uri_path, data, SECRET_KEY)
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req


# Construct the request and print the result
resp = kraken_request('/0/private/Balance', {
    "nonce": str(int(1000*time.time()))
})


print(resp.json())


NameError: name 'API_KEY' is not defined

In [31]:
resp.json()['result']['ZUSD']


'0.0042'

## Asset info

In [21]:
import requests

resp = requests.get('https://api.kraken.com/0/public/Assets?asset=XBT')

print(resp.json())


{'error': [], 'result': {'XXBT': {'aclass': 'currency', 'altname': 'XBT', 'decimals': 10, 'display_decimals': 5, 'collateral_value': 1.0, 'status': 'enabled'}}}


In [20]:
import requests

resp = requests.get('https://api.kraken.com/0/public/AssetPairs?pair=XXBTZUSD,XETHXXBT')

print(resp.json())

{'error': [], 'result': {'XETHXXBT': {'altname': 'ETHXBT', 'wsname': 'ETH/XBT', 'aclass_base': 'currency', 'base': 'XETH', 'aclass_quote': 'currency', 'quote': 'XXBT', 'lot': 'unit', 'cost_decimals': 6, 'pair_decimals': 5, 'lot_decimals': 8, 'lot_multiplier': 1, 'leverage_buy': [2, 3, 4, 5], 'leverage_sell': [2, 3, 4, 5], 'fees': [[0, 0.26], [50000, 0.24], [100000, 0.22], [250000, 0.2], [500000, 0.18], [1000000, 0.16], [2500000, 0.14], [5000000, 0.12], [10000000, 0.1], [100000000, 0.08]], 'fees_maker': [[0, 0.16], [50000, 0.14], [100000, 0.12], [250000, 0.1], [500000, 0.08], [1000000, 0.06], [2500000, 0.04], [5000000, 0.02], [10000000, 0.0], [100000000, 0.0]], 'fee_volume_currency': 'ZUSD', 'margin_call': 80, 'margin_stop': 40, 'ordermin': '0.01', 'costmin': '0.00002', 'tick_size': '0.00001', 'status': 'online', 'long_position_limit': 550, 'short_position_limit': 470}, 'XXBTZUSD': {'altname': 'XBTUSD', 'wsname': 'XBT/USD', 'aclass_base': 'currency', 'base': 'XXBT', 'aclass_quote': 'cur

## Place SELL order

In [13]:
def SELL(qty, price, symbol="XBTUSD"):
        
        resp = kraken_request('/0/private/AddOrder', {
                                                        "nonce": str(int(1000*time.time())),
                                                        "ordertype": "limit",
                                                        "type": "sell",
                                                        "volume": qty,
                                                        "pair": symbol,
                                                        "price": price
                                                    })
        
        
        return resp
        # return resp.json()['result']['ordernum']

In [23]:
resp = SELL(qty=0.002, price=22500, symbol="XBTUSD")

In [24]:
resp.json()

{'error': ['EOrder:Insufficient funds']}

## Place BUY order

In [29]:
def BUY(qty, price, symbol="XBTUSD"):
        
        resp = kraken_request('/0/private/AddOrder', {
                                                        "nonce": str(int(1000*time.time())),
                                                        "ordertype": "limit",
                                                        "type": "buy",
                                                        "volume": qty,
                                                        "pair": symbol,
                                                        "price": price
                                                    })
        
        return resp
        # return resp.json()['result']['ordernum']

In [46]:
resp = BUY(qty=0.00086, price=23000, symbol="XBTUSD")

In [45]:
0.00086956*0.997

0.00086695132

In [44]:
20/23000

0.0008695652173913044

In [47]:
resp.json()

{'error': [],
 'result': {'txid': ['O5SJJX-DPZYE-R4P7WL'],
  'descr': {'order': 'buy 0.00086000 XBTUSD @ limit 23000.0'}}}

## Calculate MA

NameError: name 'HIST_PRICE' is not defined

In [2]:
import sys
sys.path.append("S://Docs//Personal//MAEVE//MAEVE-II//3_Implementation//Scripts//")

import KrakenTrader as krkn

trader = krkn.KrakenTrader()

 # Get MAs
MA1 = trader.GET_MA(12)
MA2 = trader.GET_MA(20)

print(MA1)
print(MA2)

21861.641666666666
22167.254999999997


In [13]:
symbol="XBTUSD"
timeframe=60
MA = 20

hist_df = trader.HIST_PRICE(symbol=symbol, timeframe=timeframe)
MA1 = trader.CALC_MA(hist_df, timeperiod=MA)
maxtime = MA1['time'].max()        

print(MA1[MA1['time']==maxtime][f'MA{MA}'].values[0])

22212.215
